In [ ]:
!pip install -q datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12

#Load Model (Trained)

In [ ]:
from sentence_transformers import SentenceTransformer, SentenceTransformerTrainer, losses
from sentence_transformers.training_args import SentenceTransformerTrainingArguments

base_model = SentenceTransformer("indobenchmark/indobert-base-p1")
model = SentenceTransformer("ShoAnn/retriever_checkpoints")

config.json:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/498M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/229k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/229k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/709k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

#Load and Prepare Test Dataset

In [ ]:
#@title Compute embedding
from datasets import load_dataset
import pandas as pd
import torch

qpn_dataset = load_dataset('ShoAnn/indo_legalqa_triplets')

lb_all = pd.read_csv(
    "lb_v1_5474.csv",
    index_col=0,
    dtype={
        "pasal":"Int64",
        "angka":"Int64",
        "nomor":"Int64",
        "tahun":"Int64",
    })
queries = list(set(qpn_dataset["test"]["q"]))
corpus = []
for doc in lb_all.itertuples():
    corpus.append(f'{doc.text} >> {doc.fulltext}')
corpus_embeddings_base = base_model.encode(corpus, convert_to_tensor=True)
corpus_embeddings = model.encode(corpus, convert_to_tensor=True)

In [ ]:
#@title Prepare ground truth
ground_truth = pd.DataFrame(set(qpn_dataset["test"]["q"]), columns=["query"])
def group_pos(query):
    pos_list = []
    for i in range(len(qpn_dataset["test"]["q"])):
        if qpn_dataset["test"]["q"][i] == query:
            pos_list.append(qpn_dataset["test"]["pos"][i])
    return pos_list
ground_truth["docs"] = ground_truth["query"].apply(group_pos)
ground_truth

,query,docs
0,Kalau mahasiswa dalam melakukan demo dalam kam...,[Kegiatan yang bersifat ilmiah diselenggarakan...
1,Akhir-akhir ini sedang viral isu love scamming...,[Pasal 45A ayat (1) UU 19/2016 dapat diterapka...
2,Pasal 372 KUHP tentang apa? Berapa lama hukuma...,[Undang-Undang ini mulai berlaku setelah 3 (ti...
3,Pelaku kejahatan seksual terhadap anak dapat d...,[(5) Dalam hal tindak pidana sebagaimana dimak...
4,"Belakangan ini, beredar berita tentang kasus b...",[Undang-Undang ini mulai berlaku setelah 3 (ti...
...,...,...
443,Anak saya ditahan polisi dengan tuduhan Pasal ...,[['(1) keterangan tersangka dan atau saksi ke...
444,"Teman saya laki-laki yang sudah punya istri, m...",[Pasal 369 Pasal 483 Dengan memperhatikan rumu...
445,Apakah ada peraturan atau sistem pendidikan di...,[['(1) anak yang dijatuhi pidana penjara dite...
446,Apa hukumnya bagi seorang wanita yang baru men...,[Pasal 624 Undang-Undang ini mulai berlaku set...


In [ ]:
# Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
from tqdm import tqdm

def retrieve_docs(queries, top_k, model, corpus_embeddings):
    retrieved_docs = []
    for query in queries:
        query_embedding = model.encode(query, convert_to_tensor=True)

        similarity_scores = model.similarity(query_embedding, corpus_embeddings)[0]
        scores, indices = torch.topk(similarity_scores, k=top_k)

        docs = []
        for score, idx in zip(scores, indices):
            docs.append((corpus[idx].split(">> ")[1], score.detach().cpu().item()))
        retrieved_docs.append({"query": query, "docs": docs})
    return retrieved_docs

In [ ]:
k5_base = pd.DataFrame(retrieve_docs(queries, 5, base_model, corpus_embeddings_base))
k10_base = pd.DataFrame(retrieve_docs(queries, 10, base_model, corpus_embeddings_base))
k20_base = pd.DataFrame(retrieve_docs(queries, 20, base_model, corpus_embeddings_base))
k100_base = pd.DataFrame(retrieve_docs(queries, 100, base_model, corpus_embeddings_base))

k5 = pd.DataFrame(retrieve_docs(queries, 5, model, corpus_embeddings))
k10 = pd.DataFrame(retrieve_docs(queries, 10, model, corpus_embeddings))
k20 = pd.DataFrame(retrieve_docs(queries, 20, model, corpus_embeddings))
k100 = pd.DataFrame(retrieve_docs(queries, 100, model, corpus_embeddings))

#Compute Metrics

In [ ]:
class TestSet:
    def __init__(self, ground_truth_df, test_df):
        self.ground_truth_df = ground_truth_df
        self.test_df = test_df
    def __recall(self, row):
        query = row["query"]
        test_docs = [doc[0] for doc in row["docs"]]
        gt = self.ground_truth_df[self.ground_truth_df["query"] == query]
        gt_list = gt["docs"].iloc[0]

        tp = [doc for doc in test_docs if doc in gt_list]
        recall = len(tp) / len(gt_list) # TP / (TP + FN) where FN = len(gt_list) - len(tp)
        return recall

    def get_recall_score(self, k):
        recall = 0
        for _, row in self.test_df.iterrows():
            recall += self.__recall(row)
        return recall / len(self.test_df)

    def __reciprocal_rank(self, row):
        query = row["query"]
        test_docs = [doc[0] for doc in row["docs"]]
        gt = self.ground_truth_df[self.ground_truth_df["query"] == query]
        gt_list = gt["docs"].iloc[0]

        for rank, doc_id in enumerate(test_docs, 1):
            if doc_id in gt_list:
                return 1.0 / rank
        return 0.0

    def get_mrr_score(self, k):
        rr_sum = 0
        for _, row in self.test_df.iterrows():
            rr_sum += self.__reciprocal_rank(row)
        return rr_sum / len(self.test_df) if len(self.test_df) > 0 else 0.0


## base model

In [ ]:
test_k_5_base = TestSet(ground_truth, k5_base)
test_k_10_base = TestSet(ground_truth, k10_base)
test_k_20_base = TestSet(ground_truth, k20_base)
test_k_100_base = TestSet(ground_truth, k100_base)

recall_k_5_base = test_k_5_base.get_recall_score(5)
recall_k_10_base = test_k_10_base.get_recall_score(10)
recall_k_20_base = test_k_20_base.get_recall_score(20)
recall_k_100_base = test_k_100_base.get_recall_score(100)

print(f"Recall@5: {recall_k_5_base}")
print(f"Recall@10: {recall_k_10_base}")
print(f"Recall@20: {recall_k_20_base}")
print(f"Recall@100: {recall_k_100_base}")

mrr_k_5_base = test_k_5_base.get_mrr_score(5)
mrr_k_10_base = test_k_10_base.get_mrr_score(10)
mrr_k_20_base = test_k_20_base.get_mrr_score(20)
mrr_k_100_base = test_k_100_base.get_mrr_score(100)

print(f"mrr@5: {mrr_k_5_base}")
print(f"mrr@10: {mrr_k_10_base}")
print(f"mrr@20: {mrr_k_20_base}")
print(f"mrr@100: {mrr_k_100_base}")

Recall@5: 0.042261904761904764
Recall@10: 0.07180059523809523
Recall@20: 0.10639880952380952
Recall@100: 0.27615327380952376
mrr@5: 0.031510416666666666
mrr@10: 0.03492329223356008
mrr@20: 0.03666419961356521
mrr@100: 0.03925452557356971


## fine tuned model

In [ ]:
test_k_5 = TestSet(ground_truth, k5)
test_k_10 = TestSet(ground_truth, k10)
test_k_20 = TestSet(ground_truth, k20)
test_k_100 = TestSet(ground_truth, k100)

recall_k_5 = test_k_5.get_recall_score(5)
recall_k_10 = test_k_10.get_recall_score(10)
recall_k_20 = test_k_20.get_recall_score(20)
recall_k_100 = test_k_100.get_recall_score(100)

print(f"Recall@5: {recall_k_5}")
print(f"Recall@10: {recall_k_10}")
print(f"Recall@20: {recall_k_20}")
print(f"Recall@100: {recall_k_100}")

mrr_k_5 = test_k_5.get_mrr_score(5)
mrr_k_10 = test_k_10.get_mrr_score(10)
mrr_k_20 = test_k_20.get_mrr_score(20)
mrr_k_100 = test_k_100.get_mrr_score(100)

print(f"mrr@5: {mrr_k_5}")
print(f"mrr@10: {mrr_k_10}")
print(f"mrr@20: {mrr_k_20}")
print(f"mrr@100: {mrr_k_100}")

Recall@5: 0.004464285714285714
Recall@10: 0.004464285714285714
Recall@20: 0.010044642857142858
Recall@100: 0.0603422619047619
mrr@5: 0.0016741071428571428
mrr@10: 0.0013020833333333333
mrr@20: 0.0017189191017316016
mrr@100: 0.00227466036991027
